## 19 변수 사용성 개선

### 19.1 변수 이름 지정

### 19.2 ndarray 인스턴스 변수

In [9]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config,name)
  setattr(Config, name, value)
  try : 
    yield

  finally:
    setattr(Config, name, old_value)


In [31]:
class Variable:
  def __init__(self, data, name = None) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    # 이름 지정
    self.name = name
    self.grad = None
    self.creator = None
    #세대 수 표현
    self.generation = 0
  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1
  
  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator)
      # 중간 미분값 삭제
      if not retain_grad:
        for y in f.outputs:
          y().grad = None
  def cleargrad(self):
    self.grad = None
  #shape
  @property
  def shape(self):
    return self.data.shape
  
  @property
  def ndim(self):
    return self.data.ndim
  
  @property
  def size(self):
    return self.data.size
  #property는 메소드를 인스턴스 변수처럼 사용할 수 있음

  @property
  def dtype(self):
    return self.data.dtype
  
  def __len__(self):
    return len(self.data)
    
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n','\n'+' '*9)
    return 'variable(' + p + ')'

In [20]:
import numpy as np
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape)
x = np.array([[1,2,3],[4,5,6]])
x.shape

(2, 3)


(2, 3)

In [21]:
import weakref
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs :
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [22]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [23]:
def add(x0, x1):
  return Add()(x0,x1)

In [24]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [25]:
def square(x):
  return Square()(x)

### 19.3 len 함수와 print 함수

```
  def __len__(self):
    return len(self.data)
```
  요거 추가

In [28]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(len(x))

2


def __repr__ 추가하면 print를 할 수 있음

```
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n','\n'+' '*9)
    return 'variable(' + p + ')'
```


In [32]:
x = Variable(np.array([1,2,3]))
print(x)

variable([1 2 3])
